In [448]:
# https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

#
# This is a sample Notebook to demonstrate how to read "MNIST Dataset"
#
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)    

In [449]:
# # https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

# #
# # Verify Reading Dataset via MnistDataloader class
# #
# %matplotlib inline
# import random
# import matplotlib.pyplot as plt
# import pickle

# #
# # Set file paths based on added MNIST Datasets
# #
# input_path = 'dataset/'
# training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
# training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
# test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
# test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# #
# # Helper function to show a list of images with their relating titles
# #
# def show_images(images, title_texts):
#     cols = 5
#     rows = int(len(images)/cols) + 1
#     plt.figure(figsize=(30,20))
#     index = 1    
#     for x in zip(images, title_texts):        
#         image = x[0]        
#         title_text = x[1]
#         plt.subplot(rows, cols, index)        
#         plt.imshow(image, cmap=plt.cm.gray)
#         if (title_text != ''):
#             plt.title(title_text, fontsize = 15);        
#         index += 1

# #
# # Load MINST dataset
# #
# mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
# (data_train, label_train), (data_test, label_test) = mnist_dataloader.load_data()

# with open("dataset/pickled/data_train.pickle", "wb") as outfile:
#     pickle.dump(data_train, outfile)
# with open("dataset/pickled/label_train.pickle", "wb") as outfile:
#     pickle.dump(label_train, outfile)
# with open("dataset/pickled/data_test.pickle", "wb") as outfile:
#     pickle.dump(data_test, outfile)
# with open("dataset/pickled/label_test.pickle", "wb") as outfile:
#     pickle.dump(label_test, outfile)

# first_10_data_train = []
# first_10_label_train = []
# for i in range(0, 10):
#     first_10_data_train.append(data_train[i])
#     first_10_label_train.append(label_train[i])

# with open("dataset/pickled/first_10_data_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_data_train, outfile)
# with open("dataset/pickled/first_10_label_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_label_train, outfile)

# #
# # Show some random training and test images 
# #
# images_2_show = []
# titles_2_show = []
# for i in range(0, 10):
#    images_2_show.append(first_10_data_train[i])
#    titles_2_show.append('training image [' + str(i) + '] = ' + str(first_10_label_train[i]))    

# #for i in range(0, 5):
# #   r = random.randint(1, 10000)
# #   images_2_show.append(data_test[r])        
# #   titles_2_show.append('test image [' + str(r) + '] = ' + str(label_test[r]))    

# show_images(images_2_show, titles_2_show)


In [450]:
import random
import numpy as np

class Neuron:
    def __init__(self, numberOfOutgoing):
        self.activation = random.uniform(0, 1)
        self.bias = random.uniform(0, 1)
        self.outgoingWeights = np.array([random.uniform(0, 1) for i in range(numberOfOutgoing)])

    def cout(self):
        print("Activation: ", self.activation)
        print("Bias: ", self.bias)
        print("Outgoing Weights: ", self.outgoingWeights)


In [451]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Network:

    def __init__(self, start, first, second, end):
        self.startLayer = np.array([Neuron(first) for i in range(start)])
        self.firstLayer = np.array([Neuron(second) for i in range(first)])
        self.secondLayer = np.array([Neuron(end) for i in range(second)])
        self.endLayer = np.array([Neuron(0) for i in range(end)])

    def forwardPropagationStep(self, inLayer, outLayer):
        for neur in outLayer:
            print(neur.activation)
        print()

        # weight matrix
        weightMatrix = np.empty((outLayer.size, inLayer.size))
        for col in range (0, inLayer.size):
            for row in range (0, inLayer[0].outgoingWeights.size):
                weightMatrix[row][col] = inLayer[col].outgoingWeights[row]

        # input activation
        inActivation = np.array([neur.activation for neur in inLayer])
        
        # output biases
        outBias = np.array([neur.bias for neur in outLayer])

        outputMat = np.dot(weightMatrix, inActivation)
        outputMat = outputMat + outBias

        for i in range(outLayer.size):
            outLayer[i].activation = sigmoid(outputMat[i])

        for neur in outLayer:
            print(neur.activation)

    def clearActivations(self):
        for neur in self.startLayer:
            neur.activation = 0

        for neur in self.firstLayer:
            neur.activation = 0

        for neur in self.secondLayer:
            neur.activation = 0

        for neur in self.endLayer:
            neur.activation = 0

    # Assumes that data is between 0 and 255 value
    def prepForFullForwardPropagation(self, dataset):
        self.clearActivations()

        if len(dataset) * len(dataset[0]) != self.startLayer.size:
            print("There is a mismatch between the size of the input data and the start layer!")
            print("Size of dataset is: " + str(len(dataset) * len(dataset[0])))
            print("Size of first layer is: " + str(self.startLayer.size))

        layerIdx = 0
        for row in range(0, len(dataset)):
            for col in range(0, len(dataset[0])):
                self.startLayer[layerIdx].activation = dataset[row][col] / 255
                layerIdx += 1

    def fullForwardPropagation(self, target):
        self.forwardPropagationStep(self.startLayer, self.firstLayer)
        self.forwardPropagationStep(self.firstLayer, self.secondLayer)
        self.forwardPropagationStep(self.secondLayer, self.endLayer)
        print("Target is: " + str(target))
        print("Cost is: " + str(self.cost(target)))
    
    def cost(self, target):
        sum = 0
        for i in range(self.endLayer.size):
            if (i == target):
                sum += pow(self.endLayer[i].activation - 1.0, 2)
            else:
                sum += pow(self.endLayer[i].activation, 2)
        return sum

    def coutStartLayerActivation(self):
        for row in range(0, 28):
            for col in range(0, 28):
                print(self.startLayer[row * 28 + col].activation)
            print()

    def coutEndLayerActivation(self):
        for neur in self.endLayer:
            print(neur.activation)

    def cout(self):
        print("Start Layer")
        for neur in self.startLayer:
            neur.cout()
        print()
        print()
        print("First Layer")
        for neur in self.firstLayer:
            neur.cout()
        print()
        print()
        print("Second Layer")
        for neur in self.secondLayer:
            neur.cout()
        print()
        print()
        print("End Layer")
        for neur in self.endLayer:
            neur.cout()
        

In [452]:
# MNIST Dataset: 28 x 28 = 784
import pickle

with open("dataset/pickled/first_10_data_train.pickle", "rb") as infile:
    data_batch = pickle.load(infile)
with open("dataset/pickled/first_10_label_train.pickle", "rb") as infile:
    label_batch = pickle.load(infile)

with open("network.pickle", "rb") as infile:
   net = pickle.load(infile)

net = Network(784, 16, 16, 10)

for i in range(0, 1): #len(data_batch)):
    net.prepForFullForwardPropagation(data_batch[i])
    net.fullForwardPropagation(label_batch[i])
    print()

with open("network.pickle", "wb") as outfile:
    pickle.dump(net, outfile)

SyntaxError: expected ':' (1777750303.py, line 14)